# ProductHunt Products from the January 2023

In [322]:
import numpy as np
import pandas as pd
import gc
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from matplotlib.dates import DateFormatter
import networkx as nx
from adjustText import adjust_text
import plotly.graph_objects as go
import plotly.express as px


# Import & Analyse the raw dataset

In [323]:
# Step 1: Load the Dataset
# df = pd.read_csv("C:/Users/alime/Dropbox/PC/Documents/Coding/2023/ProductHunt_EDA_and_Unspervised_Sentiment_Analysis/Data/Stats/CSV/Stats_Year_2022.csv")
Jan_df = pd.read_csv("C:/Users/alime/Dropbox/PC/Documents/Coding/2023/ProductHunt_EDA_and_Unspervised_Sentiment_Analysis/Data/Stats/CSV/Posts_Jan_2023_Fix_Cleaned.csv")
Feb_df = pd.read_csv("C:/Users/alime/Dropbox/PC/Documents/Coding/2023/ProductHunt_EDA_and_Unspervised_Sentiment_Analysis/Data/Stats/CSV/Posts_Feb_2023_Cleaned.csv")
Mar_df = pd.read_csv("C:/Users/alime/Dropbox/PC/Documents/Coding/2023/ProductHunt_EDA_and_Unspervised_Sentiment_Analysis/Data/Stats/CSV/Posts_Mar_2023_Cleaned.csv")
# df = pd.read_csv("C:/Users/alime/Dropbox/PC/Documents/Coding/2023/ProductHunt_EDA_and_Unspervised_Sentiment_Analysis/Data/Stats/CSV/Posts_Feb_2023_Cleaned.csv")

# Assuming you have two DataFrames named 'df1' and 'df2'
df = pd.concat([Jan_df, Feb_df, Mar_df], ignore_index=True)

df.head()

,id,name,votesCount,reviewsRating,reviewsCount,commentsCount,createdAt,totalCount,topics
0,377488,Perplexity.ai,214,0.0,0,19,2023-02-01T01:10:23Z,20,Search
1,377270,Image Editor AI,46,0.0,0,11,2023-02-01T08:00:00Z,15,"Design Tools, Marketing, Tech"
2,377466,RoboHelper,21,0.0,0,5,2023-01-31T21:41:01Z,3,Productivity
3,376463,Dover Autopilot,1118,0.0,0,134,2023-01-31T08:03:56Z,20,"Hiring, Artificial Intelligence"
4,377244,Unlimited Voice Transcription with API,873,0.0,0,229,2023-01-31T08:12:31Z,20,"Productivity, Privacy"


In [324]:
df = df.drop('id', axis=1)
df.shape


(745423, 8)

In [325]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 745423 entries, 0 to 745422
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   name           745423 non-null  object 
 1   votesCount     745423 non-null  int64  
 2   reviewsRating  745423 non-null  float64
 3   reviewsCount   745423 non-null  int64  
 4   commentsCount  745423 non-null  int64  
 5   createdAt      745423 non-null  object 
 6   totalCount     745423 non-null  int64  
 7   topics         745423 non-null  object 
dtypes: float64(1), int64(4), object(3)
memory usage: 45.5+ MB


# Step 2: Data Cleaning & Manipulation

In [326]:
# Check for missing values
print(df.isnull().sum())

name             0
votesCount       0
reviewsRating    0
reviewsCount     0
commentsCount    0
createdAt        0
totalCount       0
topics           0
dtype: int64


In [327]:
df.describe()

,votesCount,reviewsRating,reviewsCount,commentsCount,totalCount
count,745423.000000,745423.0,745423.0,745423.000000,745423.000000
mean,98.227410,0.0,0.0,22.310007,6.894432
std,210.094115,0.0,0.0,61.054732,7.283637
min,1.000000,0.0,0.0,0.000000,0.000000
25%,5.000000,0.0,0.0,1.000000,0.000000
50%,30.000000,0.0,0.0,4.000000,4.000000
75%,87.000000,0.0,0.0,15.000000,12.000000
max,5022.000000,0.0,0.0,1060.000000,20.000000


### Remove the reviewRating and reviewsCount columns since they are always 0

In [328]:
df = df.drop('reviewsRating', axis=1)
df = df.drop('reviewsCount', axis=1)
df = df.drop('totalCount', axis=1)

### Remove Duplicates and keep the first appearance

In [329]:
df.drop_duplicates(subset=['name', 'createdAt'], keep='first', inplace=True)

### Change the date column type to DataTime

In [330]:
df['createdAt'] = pd.to_datetime(df['createdAt'])
df.head()

,name,votesCount,commentsCount,createdAt,topics
0,Perplexity.ai,214,19,2023-02-01 01:10:23+00:00,Search
1,Image Editor AI,46,11,2023-02-01 08:00:00+00:00,"Design Tools, Marketing, Tech"
2,RoboHelper,21,5,2023-01-31 21:41:01+00:00,Productivity
3,Dover Autopilot,1118,134,2023-01-31 08:03:56+00:00,"Hiring, Artificial Intelligence"
4,Unlimited Voice Transcription with API,873,229,2023-01-31 08:12:31+00:00,"Productivity, Privacy"


### Add a month Column

In [331]:

# Create the "Month" column with the corresponding month
df['Month'] = df['createdAt'].dt.month

# Rearrange the columns to position "Month" after "createdAt"
cols = list(df.columns)
cols.insert(cols.index('createdAt') + 1, cols.pop(cols.index('Month')))
df = df[cols]
df['Month'] = df['createdAt'].dt.strftime('%B')
df.head()


,name,votesCount,commentsCount,createdAt,Month,topics
0,Perplexity.ai,214,19,2023-02-01 01:10:23+00:00,February,Search
1,Image Editor AI,46,11,2023-02-01 08:00:00+00:00,February,"Design Tools, Marketing, Tech"
2,RoboHelper,21,5,2023-01-31 21:41:01+00:00,January,Productivity
3,Dover Autopilot,1118,134,2023-01-31 08:03:56+00:00,January,"Hiring, Artificial Intelligence"
4,Unlimited Voice Transcription with API,873,229,2023-01-31 08:12:31+00:00,January,"Productivity, Privacy"


### Adding an Hours column

In [332]:
# Convert the 'createdAt' column to pandas datetime
df['createdAt'] = pd.to_datetime(df['createdAt'])

# Create a new column 'hours' containing only the hours from the 'createdAt' column
df['hours'] = df['createdAt'].dt.hour
df['day'] = df['createdAt'].dt.day

# # Sort the DataFrame by the 'hours' column in ascending order
# New_filtered_df = df.sort_values(by='hours', ascending=True)

# New_filtered_df.head()

## Filtering the rows with months outside of Q1

In [333]:
# df = df[df['Month'] == 'February']

# List of months to keep
months_to_keep = ['January', 'February', 'March']

# Filter the DataFrame to keep only rows with the specified months
df = df[df['Month'].isin(months_to_keep)]

df.head()


,name,votesCount,commentsCount,createdAt,Month,topics,hours,day
0,Perplexity.ai,214,19,2023-02-01 01:10:23+00:00,February,Search,1,1
1,Image Editor AI,46,11,2023-02-01 08:00:00+00:00,February,"Design Tools, Marketing, Tech",8,1
2,RoboHelper,21,5,2023-01-31 21:41:01+00:00,January,Productivity,21,31
3,Dover Autopilot,1118,134,2023-01-31 08:03:56+00:00,January,"Hiring, Artificial Intelligence",8,31
4,Unlimited Voice Transcription with API,873,229,2023-01-31 08:12:31+00:00,January,"Productivity, Privacy",8,31


In [334]:
df.shape

(7876, 8)

## Usage of the Top 3 Topics over time

In [335]:
# Get the frequency of each individual topic
topic_freq = df['topics'].str.split(', ').explode().value_counts()

# Find the Top 5 most popular topics throughout months
top5_topics = topic_freq.head(3)

# Create a line plot to illustrate each topic's daily usage over time
df['createdAt'] = pd.to_datetime(df['createdAt'])
df['date'] = df['createdAt'].dt.date

fig = go.Figure()

for topic in top5_topics.index:
    topic_df = df[df['topics'].str.contains(topic)]
    daily_counts = topic_df.groupby('date').size()
    fig.add_trace(go.Scatter(x=daily_counts.index, y=daily_counts, mode='lines', name=topic))

fig.update_layout(title='Top 5 Most Popular Topics Over Time',
                  xaxis_title='Date',
                  yaxis_title='Daily Usage',
                  legend_title='Topics')

fig.show()

## Daily product creation thoughout Q1

In [338]:
import plotly.graph_objects as go

# Assuming your DataFrame is already defined as 'df'

# Group by date and count the number of products created each day
daily_product_creation = df.groupby('date').size().reset_index(name='count')

fig = go.Figure()
fig.add_trace(go.Scatter(x=daily_product_creation['date'], y=daily_product_creation['count'], mode='lines+markers'))
fig.update_layout(title='Daily Product Creation Throughout the Months', xaxis_title='Date', yaxis_title='Count')
fig.show()


## The most active days in each month

In [336]:
# Create a new column 'year-month-day' to store the date in the desired format
df['year-month-day'] = df['createdAt'].dt.strftime('%Y-%m-%d')

# Group by 'Month' and 'year-month-day' and get the count of products for each day
most_active_days = df.groupby(['Month', 'year-month-day']).size().reset_index(name='count')

# Get the index of the maximum count for each month
idx = most_active_days.groupby('Month')['count'].idxmax()

# Use the index to extract the most active day for each month
dates_to_plot = most_active_days.loc[idx, 'year-month-day'].tolist()

print(dates_to_plot)

['2023-02-28', '2023-01-31', '2023-03-22']


## Hourly product creation for the most pupular days

In [337]:
# Create a Plotly figure
fig = go.Figure()

# Variables to track the minimum and maximum counts across all dates
min_count = float('inf')
max_count = float('-inf')

# Loop through each date and add a trace to the figure
for date in dates_to_plot:
    most_active_Day_df = df[df['createdAt'].dt.date == pd.to_datetime(date).date()]
    hour_counts = most_active_Day_df['hours'].value_counts().sort_index()

    # Update min_count and max_count
    min_count = min(min_count, hour_counts.min())
    max_count = max(max_count, hour_counts.max())

    fig.add_trace(go.Scatter(x=hour_counts.index, y=hour_counts.values, mode='lines+markers', line_shape='linear', name=date))

# Calculate a suitable step size for the Y-axis ticks
step_size = max(1, round((max_count - min_count) / 10))

# Update the layout with a white background and adjust Y-axis ticks
fig.update_layout(
    title='Number of Products Launched Each Hour',
    xaxis_title='Hour of the Day',
    yaxis_title='Number of Products',
    xaxis=dict(tickvals=list(range(24))),
    yaxis=dict(tickmode='array',  # Set the tick mode to 'array' for custom tick values
               tickvals=list(range(min_count, max_count + step_size, step_size)),  # Use custom tick values
               ),
    height=500,           # Adjust the plot height to provide more space for labels
    margin=dict(t=50),    # Add margin at the top for the title
    template='plotly_white'  # Use a white background template
)

# Update marker and line styles
fig.update_traces(marker=dict(size=8, line=dict(width=2, color='black')), line=dict(width=2))

# Show the plot
fig.show()

In [340]:
# Assuming your DataFrame is already defined as 'df'

# Get the Top 5 most popular products based on VoteCount for each month
top5_products_per_month = df.groupby(df['createdAt'].dt.to_period('M')).apply(lambda x: x.nlargest(5, 'votesCount'))

top5_products_per_month[['name', 'votesCount', 'topics']].head(15)


c:\Users\alime\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arrays\datetimes.py:1162: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



name  votesCount  \
createdAt                                       
2023-01   2703    Swapped Finance        3338   
          2704        HyperSwitch        2277   
          8116              Swimm        1455   
          1811        Piggy Magic        1352   
          18052       Originality        1225   
2023-02   49019             Bento        2410   
          55913          ChainGPT        2398   
          42407              Typo        2073   
          223917           Nas.io        1936   
          65810           WebWave        1858   
2023-03   340961  Chat by Copy.ai        5022   
          340962        Chat.D-ID        2318   
          325976            GPT-4        1851   
          317683         GoIT LMS        1828   
          324152          Collato        1499   

                                                             topics  
createdAt                                                            
2023-01   2703                                 Web3, Cryptocurrency  
          2704                    API, Open Source, User Experience  
          8116          Software Engineering, Developer Tools, Tech  
          1811    Design Tools, Productivity, Artificial Intelli...  
          18052             Marketing, SEO, Artificial Intelligence  
2023-02   49019   Social Network, Social Media, Website Builder,...  
          55913           Artificial Intelligence, Web3, Blockchain  
          42407      Productivity, Analytics, SaaS, Developer Tools  
          223917                                    SaaS, Community  
          65810   Design Tools, Website Builder, No-Code, Graphi...  
2023-03   340961   Productivity, Marketing, Artificial Intelligence  
          340962            Messaging, API, Artificial Intelligence  
          325976                            Artificial Intelligence  
          317683                   Education, Tech, Online Learning  
          324152        Productivity, SaaS, Artificial Intelligence

In [341]:
# Assuming your DataFrame is already defined as 'df'

# Get the Top 5 most popular products based on VoteCount for all months
top5_products_all_months = df.nlargest(5, 'votesCount')

top5_products_all_months[['name', 'votesCount', 'topics']].head()


,name,votesCount,topics
340961,Chat by Copy.ai,5022,"Productivity, Marketing, Artificial Intelligence"
2703,Swapped Finance,3338,"Web3, Cryptocurrency"
49019,Bento,2410,"Social Network, Social Media, Website Builder,..."
55913,ChainGPT,2398,"Artificial Intelligence, Web3, Blockchain"
340962,Chat.D-ID,2318,"Messaging, API, Artificial Intelligence"
